In [ ]:
# import dependencies 
import pandas as pd 
import matplotlib.pyplot as lib 
import numpy as np

# Import the API key.
from config import weather_api_key

import time
from datetime import datetime 
from citipy import citipy

# Import the requests library.
import requests




In [2]:
# store the lat and long
# delcare each array as var 
# 6.1.4

# generate random lat 
lats = np.random.uniform(low=-90.000, high = 90.000, size = 1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size = 1500)

# create zip object in memory (packing the two together)
lat_lngs = zip(lats, lngs)
# packed into tuple (1500 of them) and access as 0,1 index form
lat_lngs


In [3]:
# unpack the lat_lngs zip object into list type
# cannot simply print out coordinates on its own without using 0,1 indexing
coordinates = list(lat_lngs)

In [4]:
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-34.75058943296823 -129.5568719077769
-38.97250318915656 167.4026430912395
33.322284142738596 135.83429785180715
-13.744192690672591 56.85509478137706
-19.645775142662515 -54.477527460729675
-44.71624776723399 -31.232394751308846
-3.435041100181806 -164.6646077662988
74.93506527281463 -146.85468584144678
-32.605183680876564 139.39932527400396
-72.33656403343323 -5.2842317880367204
78.47592698349823 113.44307570695003
-8.726488255992422 -177.5866585825404
-23.02036846480476 170.71311159761598
-74.61861423312718 -168.66934823099422
57.76080079548427 164.74229026323775
-85.7208559114662 142.43128372519368
84.72991272071638 88.43433837775956
21.768539398842933 -149.15629111395992
-16.060086850774695 -150.4659698844144
67.12060689489996 47.57256851937797
-88.5255411061992 -50.282053179093396
-84.09016703998337 157.2110830970563
49.42398523842991 5.064109394435803
-59.84985583469013 66.2319812912026
43.50462386756823 45.395290992099774
77.46801315920447 -127.98196484435519
60.86777339755895 

In [5]:
# 6.1.5 generate random wolrd cities 
# Use the citipy module to determine city based on latitude and longitude.
# iterate through the coordinates unzipped tuple 
# use citypy and add lat, long
# print the city name, chain the name to the nearest city function
# print the country name, chain the country code to nearest city function 

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
#print(cities)
len(cities)

622

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=fd1fbd1a397ad51b5a5aa7507997d4cf


In [7]:
# 1. import dependencies, initialize counters and a list to hold weather data
city_data = []
print("Beginning Data Retrieval           ")
print("-----------------------------------")

# create counters
# initialize to 1 b/c we want first iteration of logging for each response 
record_count = 1
set_count = 1

# fixed version that adds not city index, but name 
# enumerate helps refrence the index and city in list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    # remove blank spaces in city name with .replace
    city_url = url + "&q=" + city.replace(" ","+")

    # create print statement to tell us record and set counts 
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        #print(city_weather)
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval           
-----------------------------------
Processing Record 1 of Set 1 | rikitea
Processing Record 2 of Set 1 | westport
Processing Record 3 of Set 1 | shingu
Processing Record 4 of Set 1 | cap malheureux
Processing Record 5 of Set 1 | camapua
Processing Record 6 of Set 1 | cidreira
Processing Record 7 of Set 1 | samusu
City not found. Skipping...
Processing Record 8 of Set 1 | college
Processing Record 9 of Set 1 | port pirie
Processing Record 10 of Set 1 | cape town
Processing Record 11 of Set 1 | saskylakh
Processing Record 12 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 13 of Set 1 | tadine
Processing Record 14 of Set 1 | vaini
Processing Record 15 of Set 1 | ossora
Processing Record 16 of Set 1 | hobart
Processing Record 17 of Set 1 | dikson
Processing Record 18 of Set 1 | hilo
Processing Record 19 of Set 1 | fare
Processing Record 20 of Set 1 | leshukonskoye
Processing Record 21 of Set 1 | ushuaia
Processing Record 22 of Set 1 | b

Processing Record 39 of Set 4 | tumannyy
City not found. Skipping...
Processing Record 40 of Set 4 | linqiong
Processing Record 41 of Set 4 | kuche
City not found. Skipping...
Processing Record 42 of Set 4 | hamilton
Processing Record 43 of Set 4 | grand river south east
City not found. Skipping...
Processing Record 44 of Set 4 | palabuhanratu
City not found. Skipping...
Processing Record 45 of Set 4 | tiznit
Processing Record 46 of Set 4 | querecotillo
Processing Record 47 of Set 4 | muros
Processing Record 48 of Set 4 | bolungarvik
City not found. Skipping...
Processing Record 49 of Set 4 | nyimba
Processing Record 50 of Set 4 | lakes entrance
Processing Record 1 of Set 5 | formoso do araguaia
City not found. Skipping...
Processing Record 2 of Set 5 | clyde river
Processing Record 3 of Set 5 | ahar
Processing Record 4 of Set 5 | utiroa
City not found. Skipping...
Processing Record 5 of Set 5 | vila franca do campo
Processing Record 6 of Set 5 | bonavista
Processing Record 7 of Set 5 

Processing Record 26 of Set 8 | lucapa
Processing Record 27 of Set 8 | saint-francois
Processing Record 28 of Set 8 | langgons
Processing Record 29 of Set 8 | mount isa
Processing Record 30 of Set 8 | piacabucu
Processing Record 31 of Set 8 | trairi
Processing Record 32 of Set 8 | galle
Processing Record 33 of Set 8 | acarau
Processing Record 34 of Set 8 | ponta delgada
Processing Record 35 of Set 8 | geraldton
Processing Record 36 of Set 8 | chekhov
Processing Record 37 of Set 8 | bathsheba
Processing Record 38 of Set 8 | otjiwarongo
Processing Record 39 of Set 8 | harbour breton
Processing Record 40 of Set 8 | waipawa
Processing Record 41 of Set 8 | jinhua
Processing Record 42 of Set 8 | sebring
Processing Record 43 of Set 8 | kovdor
Processing Record 44 of Set 8 | henties bay
Processing Record 45 of Set 8 | nakhon phanom
Processing Record 46 of Set 8 | chumikan
Processing Record 47 of Set 8 | saint augustine
Processing Record 48 of Set 8 | ilulissat
Processing Record 49 of Set 8 | k

Processing Record 14 of Set 12 | yugawara
Processing Record 15 of Set 12 | kaduna
Processing Record 16 of Set 12 | emerald
Processing Record 17 of Set 12 | berdigestyakh
Processing Record 18 of Set 12 | batagay
Processing Record 19 of Set 12 | paredon
Processing Record 20 of Set 12 | grand gaube
Processing Record 21 of Set 12 | buraydah
Processing Record 22 of Set 12 | kutum
Processing Record 23 of Set 12 | portland
Processing Record 24 of Set 12 | san jose
Processing Record 25 of Set 12 | mnogovershinnyy
Processing Record 26 of Set 12 | sutton
Processing Record 27 of Set 12 | pierre
Processing Record 28 of Set 12 | kingsville
Processing Record 29 of Set 12 | belaya gora
Processing Record 30 of Set 12 | karkaralinsk
City not found. Skipping...
Processing Record 31 of Set 12 | vardo
Processing Record 32 of Set 12 | ahuimanu
Processing Record 33 of Set 12 | tuggurt
City not found. Skipping...
Processing Record 34 of Set 12 | abonnema
Processing Record 35 of Set 12 | coihaique
Processing 

In [11]:
# 6.2.7 create the df for city weather data
# now we have a list of dict, use pandas to make our df
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
print(city_data)
#city_data_df.head(10)

[{'City': 'Rikitea', 'Lat': -23.1203, 'Lng': -134.9692, 'Max Temp': 76.69, 'Humidity': 70, 'Cloudiness': 41, 'Wind Speed': 11.03, 'Country': 'PF', 'Date': '2022-05-10 00:53:44'}, {'City': 'Westport', 'Lat': 41.1415, 'Lng': -73.3579, 'Max Temp': 59.45, 'Humidity': 25, 'Cloudiness': 0, 'Wind Speed': 14.52, 'Country': 'US', 'Date': '2022-05-10 00:53:44'}, {'City': 'Shingu', 'Lat': 33.7333, 'Lng': 135.9833, 'Max Temp': 59.86, 'Humidity': 92, 'Cloudiness': 100, 'Wind Speed': 7.23, 'Country': 'JP', 'Date': '2022-05-10 00:53:45'}, {'City': 'Cap Malheureux', 'Lat': -19.9842, 'Lng': 57.6142, 'Max Temp': 73.47, 'Humidity': 89, 'Cloudiness': 22, 'Wind Speed': 3, 'Country': 'MU', 'Date': '2022-05-10 00:50:24'}, {'City': 'Camapua', 'Lat': -19.5314, 'Lng': -54.0439, 'Max Temp': 71.11, 'Humidity': 52, 'Cloudiness': 5, 'Wind Speed': 3.44, 'Country': 'BR', 'Date': '2022-05-10 00:53:46'}, {'City': 'Cidreira', 'Lat': -30.1811, 'Lng': -50.2056, 'Max Temp': 67.14, 'Humidity': 85, 'Cloudiness': 5, 'Wind Spe

In [9]:
# 6.3.1 plot lat vs temp
# Extract relevant fields from the DataFrame for plotting.
lats = city_data_df["Lat"]
max_temps = city_data_df["Max Temp"]
humidity = city_data_df["Humidity"]
cloudiness = city_data_df["Cloudiness"]
wind_speed = city_data_df["Wind Speed"]

In [10]:

# Build the scatter plot for latitude vs. max temperature.
plt.scatter(lats,
            max_temps,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure.
#plt.savefig("weather_data/Fig1.png")

# Show plot.
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# 6.3.2 plot lat vs humidity
# Build the scatter plots for latitude vs. humidity.
plt.scatter(lats,
            humidity,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Humidity "+ time.strftime("%x"))
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig2.png")
# Show plot.
plt.show()

In [ ]:
# 6.3.3 Build the scatter plots for latitude vs. cloudiness.
plt.scatter(lats,
            cloudiness,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Cloudiness (%) "+ time.strftime("%x"))
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig3.png")
# Show plot.
plt.show()

In [ ]:
# 6.3.4 Build the scatter plots for latitude vs. wind speed.
plt.scatter(lats,
            wind_speed,
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties.
plt.title(f"City Latitude vs. Wind Speed "+ time.strftime("%x"))
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)
# Save the figure.
plt.savefig("weather_data/Fig4.png")
# Show plot.
plt.show()